In [1]:
import torch
import torch.nn as nn
from transformers import ElectraTokenizer, ElectraForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from tqdm import tqdm 

In [2]:
# koELECTRA 토크나이저 불러오기
tokenizer = ElectraTokenizer.from_pretrained("koelectra-base-v3-discriminator")

In [4]:
''' 모델가중치를 불러오거나 체크포인트를 불러올 때 실행 '''

# 모델의 상태 딕셔너리를 로드합니다.
model_state_dict = torch.load("Ternary_model_state_dict_epoch_15.pt")

# 모델을 생성하고 상태를 로드합니다.
model = ElectraForSequenceClassification.from_pretrained("koelectra-base-v3-discriminator", num_labels=3)
model.load_state_dict(model_state_dict)

# 옵티마이저의 상태 딕셔너리를 로드합니다.
optimizer_state_dict = torch.load("Ternary_optimizer_state_dict_epoch_15.pt")

Some weights of the model checkpoint at koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense

In [5]:
# 변경하고자 하는 Dropout 비율
new_dropout_rate = 0.2

# 모든 Dropout 레이어의 비율 변경
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Dropout):
        module.p = new_dropout_rate


In [22]:
# 모델 구조 확인
print(model)

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.2, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [6]:
# 장치 설정 (GPU 사용을 위해)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

## 실제로 예측해보기

In [7]:
# 한국어 문장을 입력으로 받아서 예측 라벨을 출력하는 함수
def predict_label(sentence, model, tokenizer, device):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=1).item()
        return predicted_label

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("../dataset/2018산림복지_본문_맞춤법검사.txt", encoding='UTF-8')
df

,sentence,predicted_sentiment
0,이제는 열심히 부지런히 최선을 잘 봉사하는 삶을 내려놓아도 잘 산다고 해서 대충 살...,부정
1,필리핀 단기선교를 시작으로 2020년 코로나 여파에도 나의 삶은 여전히 열심히 부지...,부정
2,교육 수료증 5개 임산물 가공 산야초 산림경영 회복적 교사 심리학 국가자격증 1개 ...,긍정
3,뭐든 열심히 하는 나에게 맞는일맞는 일더위에도 재미있었고 사람을 멀리하는 나에게 관...,긍정
4,오지랖이 넓어 실수도 많이 있었지! 교육청에서 좋은 분들도 만났고 과장님께서는 늘 ...,긍정
...,...,...
164988,더불어 치병 중인 블로그 이웃님들과도 파이팅을 함께 외쳐본다.,부정
164989,다음은 2020년 1월 7일부터 20일까지의 일기다,부정
164990,올해 열여섯 살이 된 반려묘 냥이 하루 종일 토하던 날 기운 없이 앉아 있는 고양이...,긍정
164991,2020년 1월 7일 하루 종일 비가 내린다.,부정


In [15]:
# 한국어 문장 입력 받기
korean_sentences = df['correct_str'].tolist()

# 예측 라벨 출력
emotion_labels = {0:'부정', 1:'중립', 2:'긍정'}
predicted_label = [emotion_labels[predict_label(korean_sentence, model, tokenizer, device)] for korean_sentence in tqdm(korean_sentences)]
df['label'] = predicted_label

100%|██████████| 135824/135824 [1:38:11<00:00, 23.05it/s] 


In [20]:
ddff = df[['correct_str','label']]
ddff

,correct_str,label
0,블로그 mom 5개의 글 mom 목록 열기 영어 명언 모음입니다.,긍정
1,"mom 2018, 12, 31, 19, 52 https blog NAVER com ...",긍정
2,An enemy generally says and believes what he w...,부정
3,True love is the joy of life 진실한 사랑은 인생의 환희다,긍정
4,Crape idem 현재를 즐겨라.,긍정
...,...,...
135819,한편 경계의 의미로 이 사자성어를 추천한 이들도 눈에 띈다.,부정
135820,조은영 원광대 교수 미술과는 2017년을 종합하기에는 수락 석출 외의 단어들이 지나...,부정
135821,올해의 사자성어는 3위부터 5위까지는 약 16대의 고른 분포를 보인 것이 특징이다.,긍정
135822,"4위는 16, 5위는 15, 1 쳤다",부정


In [22]:
ddff['label'].value_counts()

긍정    94869
부정    32320
중립     8635
Name: label, dtype: int64

In [21]:
ddff[ddff['label'] == '중립']

,correct_str,label
6,어떠세요.,중립
24,WHO 비교할 LED 달라 만들기 기기 전문 이바지할 계절 최근 3 phosphor...,중립
37,제안하세요.,중립
62,속에 있는 얘기 남겨주세요.,중립
64,속에 있는 얘기 남겨주세요.,중립
...,...,...
135712,새해 복 많이 받으세요.,중립
135724,편의점 한의원 약국 학원 부동산 운동센터 치과 패스트푸드점 병원 등등 고민하다 늦지...,중립
135793,아무리 추이를 따르더라도 하실 일은 하셔야지요.,중립
135795,소는 밭을소는 밭을 갈고 말은 사람이 타는 법이지요.,중립


In [23]:
ddff.rename(columns={'correct_str': 'sentence'}, inplace=True)
ddff.rename(columns={'label': 'predicted_sentiment'}, inplace=True)
ddff.to_csv("./Ternary_classification_2018.csv", index=False, encoding='UTF-8')

c:\Users\user\anaconda3\envs\NIFOS\lib\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
